In [43]:
# TODO: 1. Add or remove some features
#       2. Explore the data and find some interesting patterns

In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [76]:
df = pd.read_csv(r'/home/max/Desktop/kaggle/house_prise/Archive/train.csv')

In [106]:
pd.Series(df.isna().sum(),index=df.columns).loc[lambda x: x>0].sort_values(ascending=False)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
MasVnrType       872
FireplaceQu      690
LotFrontage      259
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
BsmtExposure      38
BsmtFinType2      38
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
MasVnrArea         8
Electrical         1
dtype: int64

In [ ]:
df = df.drop(columns='Id')
df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].median())
df['Street'] = df['Street'].map({'Grvl': -1, 'Pave': 1})
df['Alley'] = df['Alley'].map({'Grvl': -1, 'Pave': 1, np.nan: 0})
df['LotFrontage_Street'] = df['LotFrontage'] * df['Street']
df['LotFrontage_Alley'] = df['LotFrontage'] * df['Alley']
df['LotShape'] = df['LotShape'].map({'Reg': -1, 'IR3': 1, 'IR2': 2, 'IR1': 3})
df['LotArea_LotShape'] = df['LotArea'] * df['LotShape']
df['']

In [46]:
def clean_data(df):
    # Drop column: 'Id', 'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu'
    df = df.drop(columns=['Id', 'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu'])
    # Replace missing values with the median of each column in: 'LotFrontage', 'MasVnrArea'
    df = df.fillna({'LotFrontage': df['LotFrontage'].median(), 'MasVnrArea': df['MasVnrArea'].median()})
    # Replace missing values with "None"
    df = df.fillna({'GarageType': "None", 'GarageFinish': "None", 
                    'GarageQual': "None", 'GarageCond': "None", 
                    'BsmtCond': "None", 'BsmtQual': "None", 
                    'BsmtExposure': "None", 'BsmtFinType1': "None", 
                    'BsmtFinType2': "None", 'MasVnrType': "None"})
    # Replace missing values with 0 in column: 'GarageYrBlt'
    df = df.fillna({'GarageYrBlt': 0})
    # Replace missing values with the most common value of each column in: 'Electrical'
    df = df.fillna({'Electrical': df['Electrical'].mode()[0]})
    return df

df = clean_data(df)
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [47]:
df = pd.get_dummies(df)

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 280 entries, MSSubClass to SaleCondition_Partial
dtypes: bool(243), float64(3), int64(34)
memory usage: 768.6 KB


In [49]:
num_cols = df.select_dtypes(include='number').columns
obj_cols = df.select_dtypes(exclude='number').columns

In [50]:
df[obj_cols] = df[obj_cols].astype(int)

In [74]:
correlation = df.corr()
saleprice_corr = correlation['SalePrice'].sort_values(ascending=False)
print(saleprice_corr[['Utilities_AllPub', 'Utilities_NoSeWa']])

Utilities_AllPub    0.014314
Utilities_NoSeWa   -0.014314
Name: SalePrice, dtype: float64


In [52]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_log_error

X = df.drop(columns='SalePrice')
y = df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
y_true = y_test

rmse = root_mean_squared_log_error(y_true, y_pred)
print(f'RMSE: {rmse}')

RMSE: 0.1498697129612699


In [53]:
feature_importance = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
print(feature_importance)

OverallQual           5.494409e-01
GrLivArea             1.281392e-01
TotalBsmtSF           3.456638e-02
2ndFlrSF              3.166398e-02
1stFlrSF              2.743893e-02
                          ...     
Condition2_RRAe       1.632340e-09
Electrical_Mix        0.000000e+00
Exterior2nd_CBlock    0.000000e+00
RoofMatl_Membran      0.000000e+00
Utilities_NoSeWa      0.000000e+00
Length: 279, dtype: float64


In [54]:
# from sklearn.feature_selection import mutual_info_regression
# # Feature importance
# feature_importances = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)

# # Mutual information
# mutual_info = mutual_info_regression(X, y)
# mutual_info_series = pd.Series(mutual_info, index=X.columns).sort_values(ascending=False)

# # Top features
# top_features_importance = feature_importances.head(20)
# top_features_mutual_info = mutual_info_series.head(20)

# # Create interaction terms for top features
# selected_features = top_features_importance.index.union(top_features_mutual_info.index)
# for feature1 in selected_features:
#     for feature2 in selected_features:
#         if feature1 != feature2:
#             df[f'{feature1}_x_{feature2}'] = df[feature1] * df[feature2]

In [55]:
# X = df.drop(columns='SalePrice')
# y = df['SalePrice']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# rf = RandomForestRegressor()
# rf.fit(X_train, y_train)
# y_pred = rf.predict(X_test)
# y_true = y_test

# rmse = root_mean_squared_log_error(y_true, y_pred)
# print(f'RMSE: {rmse}')

In [56]:
# feature_importances = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
# print(feature_importances.head(20))

In [57]:
# df.columns

In [58]:
# def draw_hist(col: str):
#     plt.figure(figsize=(12, 6))
#     sns.histplot(df[col], kde=True, bins=50, color='blue', alpha=0.5)